In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as func
from pyspark.sql.types import DateType, StringType
from datetime import datetime
from pyspark.sql.functions import date_format
import os
import sys
import psycopg2
import pandas as pd
from os import listdir
from os.path import isfile, join
import re
import matplotlib.pyplot as plot

In [ ]:
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

In [ ]:
spark = SparkSession.builder.appName('Covid-19').getOrCreate()
df_covid = spark.read.json("test-2022-05-04-09-41-41")
df_covid_deaths = df_covid.select(df_covid.date, df_covid.death).filter(func.col("death").isNotNull())
df_covid_deaths = df_covid_deaths.withColumn("date", df_covid_deaths["date"].cast(StringType()))
df_covid_deaths.show()

In [ ]:
date_format_udf =  func.udf (lambda x: datetime.strptime(x, '%Y%m%d'), DateType())
df = df_covid_deaths.withColumn("date", date_format(date_format_udf(func.col('date')), 'MM-dd-yyy'))
df.show()

In [ ]:
df.write.mode("overwrite").csv(path="ClimateChangeModified/")

In [ ]:
# driver = "org.postgresql.Driver"
# url = ""
# table = "deathtodate"
# user = ""
# password = ""
# db_name = ""
#
# files = [f for f in listdir("ClimateChangeModified") if isfile(join("ClimateChangeModified", f))]
#
# for f in files:
#     if re.search(".csv$", f):
#         try:
#             conn = psycopg2.connect(f"host='{url}' port='5432' dbname='{db_name}' user='{user}' password='{password}'")
#             cur = conn.cursor()
#             f = open(r'ClimateChangeModified/' + f, 'r')
#             cur.copy_from(f, table, sep=',', columns=["date", "death"])
#             conn.commit()
#             conn.close()
#             print("Success")
#         except:
#             print("Error With Connecting Server")

In [ ]:
files = [f for f in listdir("ClimateChangeModified") if isfile(join("ClimateChangeModified", f))]

for f in files:
    if re.search(".csv$", f):
        dataFrame = pd.read_csv("ClimateChangeModified/" + f, names=["date", "death"])

In [ ]:
dataFrame['date']= pd.to_datetime(dataFrame['date'])
dataFrame.plot(x="date", y="death", kind="area")